# Milestone 1

In this milestone, we are using the Figshare API to pull data and analyze it in upcoming milestones.

## Downloading the data from Figshare

In [ ]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [ ]:
# Daily rainfall over NSW, Australia
# https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681
article_id = 14096681
# Metadata for the download
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data"

In [ ]:
# List files in the associated dataset
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

In [ ]:
# Retrieve `data.zip`
files_to_dl = ["data.zip"]
for f in files:
    if f["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(f["download_url"], f"{output_directory}/{f['name']}")

In [ ]:
# Extract `data.zip`
output_zip_file = os.path.join(output_directory, "./data.zip")
with zipfile.ZipFile(output_zip_file, 'r') as f:
    f.extractall(output_directory)

## Combine CSV files

In [ ]:
# Gather a list of files of CSV to merge
files = glob.glob(f'{output_directory}/*.csv')
files.remove(f'{output_directory}/observed_daily_rainfall_SYD.csv')
# files = files[0:1]
files

In [ ]:
columns_to_merge = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]

combined_path = f'{output_directory}/combined_data.csv'

In [ ]:
%%time

# files = glob.glob('dailyrainfall/*.csv')
# df = pd.concat((pd.read_csv(files, index_col=0)
#                 .assign(model=re.findall("/([^_]*)", file)[0])
#                 for file in files)
#               )

# A Pythonic way (but not the most memory-efficient way) for merging the data
df = pd.concat((pd.read_csv(f, index_col=0, usecols=columns_to_merge)
                .assign(model=f[len(output_directory)+1:-len("_daily_rainfall_NSW.csv")])
                for f in files)
              )
df.to_csv(combined_path)

In [ ]:
%%sh
du -sh ../data/combined_data.csv

In [ ]:
print(df.shape)

In [ ]:
df.head()

> | Team Member  | Operating System | RAM  | Processor         | Is SSD | Time taken  |
> |:------------:|:----------------:|:----:|:-----------------:|:------:|:-----------:|
> | Chen, Ziyi   | OSX 13.2.1       | 32GB | M1 (10 processors)| YES    |  3min 30s   |
> | Guron, Mike  | Windows 11       | 16GB | Intel i7-12700H   | YES    |  5min 56s   |
> | Raina, Roan  | macOS 13.2.1     | 16GB | M2 (8 core)       | YES    |  3min 17s   |
> | Wong, Kelvin | Linux Mint 21    | 16GB | AMD Ryzen 5 3500U | YES    |  10min 6s   |
> 
> Table 1: Time taken to combine the CSV files

## EDA

### Baseline

This is the baseline time needed to load the CSV file as-is.

In [ ]:
%%time
df = pd.read_csv(f"{output_directory}/combined_data.csv")

In [ ]:
df.info()

> | Team Member  | Operating System | RAM  | Processor         | Is SSD | Time taken    | Memory usage |
> |:------------:|:----------------:|:----:|:-----------------:|:------:|:-------------:|:------------:|
> | Chen, Ziyi   | OSX 13.2.1       | 32GB | M1 (10 processors)| YES    | 34s           | 3.3+ GB      |
> | Guron, Mike  | Windows 11       | 16GB | Intel i7-12700H   | YES    | 1min 16s      | 3.3+ GB      |
> | Raina, Roan  | macOS 13.2.1     | 16GB | M2 (8 core)       | YES    | 31.6s         | 3.3+ GB      |
> | Wong, Kelvin | Linux Mint 21    | 16GB | AMD Ryzen 5 3500U | YES    | 2min 45s      | 3.3+ GB      |
> 
> Table 2: Time taken to read the combined CSV (baseline)

#### Observations from Baseline

(WIP)

### Approach 1: Change the `dtype` of the data

We notice that by default it uses `float64` if we do not specify it. First, we try to see if switching to `float32` would make a smaller memory footprint, as well as a faster time.

In [ ]:
%%time
df_float32 = pd.read_csv(f"{output_directory}/combined_data.csv", dtype={
    'lat_min': 'float32',
    'lat_max': 'float32',
    'lon_min': 'float32',
    'lon_max': 'float32',
    'rain (mm/day)': 'float32'
})

In [ ]:
df_float32.info()

> | Team Member  | Operating System | RAM  | Processor         | Is SSD | Time taken    | Memory usage |
> |:------------:|:----------------:|:----:|:-----------------:|:------:|:-------------:|:------------:|
> | Chen, Ziyi   | OSX 13.2.1       | 32GB | M1 (10 processors)| YES    | 31.7s         | 2.1+ GB      |
> | Guron, Mike  | Windows 11       | 16GB | Intel i7-12700H   | YES    | 1min 11s      | 2.1+ GB      |
> | Raina, Roan  | macOS 13.2.1     | 16GB | M2 (8 core)       | YES    | 30.5s         | 2.1+ GB      |
> | Wong, Kelvin | Linux Mint 21    | 16GB | AMD Ryzen 5 3500U | YES    | 2min 28s      | 2.1+ GB      |
> 
> Table 3: Time taken to read the combined CSV (approach 1: use `float32` instead of `float64`)

#### Observations from Approach 1

(WIP)

### Approach 2: Load only column(s) we want

The dataset contains a number of columns that we may not need to use in one go. In this approach, we try to just load one column from the combined CSV file.

In [ ]:
%%time
df_only_rain = pd.read_csv(f"{output_directory}/combined_data.csv", usecols=["rain (mm/day)"])

In [ ]:
df_only_rain.info()

> | Team Member  | Operating System | RAM  | Processor         | Is SSD | Time taken    | Memory usage |
> |:------------:|:----------------:|:----:|:-----------------:|:------:|:-------------:|:------------:|
> | Chen, Ziyi   | OSX 13.2.1       | 32GB | M1 (10 processors)| YES    | 16.1s         | 476.6 MB     |
> | Guron, Mike  | Windows 11       | 16GB | Intel i7-12700H   | YES    | 37.1s         | 476.6 MB     |
> | Raina, Roan  | macOS 13.2.1     | 16GB | M2 (8 core)       | YES    | 14.9s         | 476.6 MB     |
> | Wong, Kelvin | Linux Mint 21    | 16GB | AMD Ryzen 5 3500U | YES    | 1min 6s       | 476.6 MB     |
> 
> Table 4: Time taken to read the combined CSV (approach 2: just load `rain (mm/day)`)

#### Observations from Approach 2

(WIP)

## EDA in R

Here, we explore the EDA in R instead of Python. We try "exporting" our data frame as a Parquet file for processing in R.

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%time
df.to_parquet(f"{output_directory}/combined_data.parquet")

> Why we choose Parquet?
> 
> (WIP)

In [ ]:
%%R
library(dplyr)
library(arrow)

In [ ]:
%%R
r_parquet <- open_dataset("../data/combined_data.parquet")
r_df <- r_parquet |> collect()

In [ ]:
%%time
%%R
r_df |> str()

In [ ]:
%%time
%%R
r_df |> summary()

In [ ]:
%%time
%%R
r_df |> head()

In [ ]:
%%time
%%R
r_df |> tail()